In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("./input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

['.DS_Store', 'feature_report.csv', 'sample_submission.csv', 'test.csv', 'test_leak_org.csv', 'test_leak_paradox.csv', 'test_leak_paradox_extend.csv', 'train.csv', 'train_leak_org.csv', 'train_leak_paradox.csv', 'train_leak_paradox_extend.csv']


In [3]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [4]:
test["target"] = train["target"].mean()

In [5]:
paradox_cols = ['ced6a7e91', '9df4daa99', '83c3779bf', 'edc84139a', 'f1e0ada11', 
                '73687e512', 'aa164b93b', '342e7eb03', 'cd24eae8a', '8f3740670', 
                '2b2a10857', 'a00adf70e', '3a48a2cd2', 'a396ceeb9', '9280f3d04', 
                'fec5eaf1a', '5b943716b', '22ed6dba3', '5547d6e11', 'e222309b0', 
                '5d3b81ef8', '1184df5c2', '2288333b4', 'f39074b55', 'a8b721722', 
                '13ee58af1', 'fb387ea33', '4da206d28', 'ea4046b8d', 'ef30f6be5', 
                'b85fa8b27', '2155f5e16', '794e93ca6', '070f95c99', '939f628a7', 
                '7e814a30d', 'a6e871369', '0dc4d6c7d', 'bc70cbc26', 'aca228668']

paradox_rows_name = ['a70090dc7','5b74875d4','ca6c4ea0d','242407fc6','7ff66b22b',
                     'f43a48254','c9573f4ac','c14136677','08f544828','e77e32574',
                     '61636fa41','4c0316d32','414871b56','2c518cd87','9eb93baef',
                     'd63b2307d','db9ddeb12','7e45f5bd5','680ecfac8','814f58e7b',
                     'eb850ef06','6edaf114d']

paradox_rows = []
for prn in paradox_rows_name:
    paradox_rows.append(train[train.ID == prn].index[0])

#tmp = train.loc[paradox_rows, ["ID","target"] + paradox_cols]
#print('shape', tmp.shape)
#tmp

In [92]:
new_col_last_1 = ['fec5644cf', 'caa9883f6', '9437d8b64', '68811ba58', 'ef4b87773',
        'ff558c2f2', '8d918c64f', '0b8e10df6', '2d6565ce2', '0fe78acfa',
        'b75aa754d', '2ab9356a0', '4e86dd8f3', '348aedc21', 'd7568383a',
        '856856d94', '69900c0d1', '02c21443c', '5190d6dca', '20551fa5b',
        '79cc300c7', '8d8276242', 'da22ed2b8', '89cebceab', 'f171b61af',
        '3a07a8939', '129fe0263', 'e5b2d137a', 'aa7223176', '5ac7e84c4',
        '9bd66acf6', '4c938629c', 'e62c5ac64', '57535b55a', 'a1a0084e3',
        '2a3763e18', '474a9ec54', '0741f3757', '4fe8b17c2', 'd5754aa08']
new_col_last_2 =['920a04ee2', '93efdb50f', '15ea45005', '78c57d7cd', '91570fb11',
        'c5dacc85b', '145c7b018', '590b24ab1', 'c283d4609', 'e8bd579ae',
        '7298ca1ef', 'ce53d1a35', 'a8f80f111', '2a9fed806', 'feb40ad9f',
        'cfd255ee3', '31015eaab', '303572ae2', 'cd15bb515', 'cb5161856',
        'a65b73c87', '71d64e3f7', 'ec5fb550f', '4af2493b6', '18b4fa3f5',
        '3d655b0ed', '5cc9b6615', '88c0ec0a6', '8722f33bb', '5ed0c24d0',
        '54f26ee08', '04ecdcbb3', 'ade8a5a19', 'd5efae759', 'ac7a97382',
        'e1b20c3a6', 'b0fcfeab8', '438b8b599', '43782ef36', 'df69cf626']
new_col_last_3 = ['5b465f819', 'a2aa0e4e9', '944e05d50', '4f8b27b6b', 'a498f253f',
        'c73c31769', '025dea3b3', '616c01612', 'f3316966c', '83ea288de',
        '2dbeac1de', '47b7b878f', 'b4d41b335', '686d60d8a', '6dcd9e752',
        '7210546b2', '78edb3f13', '7f9d59cb3', '30992dccd', '26144d11f',
        'a970277f9', '0aea1fd67', 'dc528471e', 'd51d10e38', 'efa99ed98',
        '48420ad48', '7f38dafa6', '1af4ab267', '3a13ed79a', '73445227e',
        '971631b2d', '57c4c03f6', '7f91dc936', '0784536d6', 'c3c3f66ff',
        '052a76b0f', 'ffb34b926', '9d4f88c7b', '442b180b6', '948e00a8d']
new_col_last_4 = ['c13ee1dc9', 'abb30bd35', 'd2919256b', '66728cc11', 'eab8abf7a',
       'cc03b5217', '317ee395d', '38a92f707', '467c54d35', 'e8f065c9d',
       '2ac62cba5', '6495d8c77', '94cdda53f', '13f2607e4', '1c047a8ce',
       '28a5ad41a', '05cc08c11', 'b0cdc345e', '38f49406e', '773180cf6',
       '1906a5c7e', 'c104aeb2e', '8e028d2d2', '0dc333fa1', '28a785c08',
       '03ee30b8e', '8e5a41c43', '67102168f', '8b5c0fb4e', '14a22ab1a',
       '9fc776466', '4aafb7383', '8e1dfcb94', '55741d46d', '8f940cb1b',
       '758a9ab0e', 'fd812d7e0', '4ea447064', '6562e2a2c', '343922109']

new_col_last_5 = ['8f6514df0', '6679fe54f', '5e62457b7', 'f17ff4efd', 'ec7f7017f',
       'c02ab7d25', '8c309c553', 'e0b968d7b', '22b980fc8', '3b6b46221',
       '3e4a6796d', 'c680e9350', '834fb292d', 'e3d33877c', '4052a9419',
       'b95be4138', '16517c8b0', '219e051b5', 'a6fbe0987', '37d7af8ad',
       'b84b2f72d', '775577e6f', '4f0c5f900', 'a68b83290', '2a2832b07',
       'ce1f5b02a', 'a6c9347a7', '82c9b4fcd', '7f78a36f7', 'f49ff3269',
       '89cffafe9', 'aeb3a6ccf', 'c7753cbfc', '4d6a1439e', '2123a4f36',
       '5c56fccf1', '03bfe48b2', '6beb0b35d', '9fb38aabe', 'ae141696e']

new_col_last_6 = ['266525925', '4b6dfc880', '2cff4bf0c', 'a3382e205', '6488c8200',
       '547d3135b', 'b46191036', '453128993', '2599a7eb7', '2fc60d4d9',
       '009319104', 'de14e7687', 'aa31dd768', '2b54cddfd', 'a67d02050',
       '37aab1168', '939cc02f5', '31f72667c', '6f951302c', '54723be01',
       '4681de4fd', '8bd53906a', '435f27009', 'f82167572', 'd428161d9',
       '9015ac21d', 'ec4dc7883', '22c7b00ef', 'd4cc42c3d', '1351bf96e',
       '1e8801477', 'b7d59d3b5', 'a459b5f7d', '580f5ff06', '39b3c553a',
       '1eec37deb', '692c44993', 'ce8ce671e', '88ef1d9a8', 'bf042d928']

In [7]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1',
       '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
       'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', 
       '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212',  '66ace2992',
       'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', 
       '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a',
       '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2',  '0572565c2',
       '190db8488',  'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98']

In [8]:
##extra_feats should be a list of feature lists


def get_log_pred(data, feats, extra_feats, lag=0):
    offset = lag + 2
    f1 = feats[:(offset * -1)]
    f2 = feats[offset:]
    for ef in extra_feats:
        f1 += ef[:(offset * -1)]
        f2 += ef[offset:]
    
    d1 = data[f1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = data[f2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = data[feats[offset-2]]
    d2 = d2[d2['pred'] != 0] # Keep?
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    d4 = d1[~d1.duplicated(['key'], keep=False)]
    d5 = d4.merge(d3, how='inner', on='key')
    
    d = d1.merge(d5, how='left', on='key')
    return d.pred.fillna(0)


In [104]:
#def _get_leak(df, cols, lag=0):
#    d1 = df[cols[:-lag-2]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
#    d2 = df[cols[lag+2:]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
#    d2['pred'] = df[cols[lag]]
#    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
#    d3 = d2[~d2.duplicated(['key'], keep=False)]
#    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [128]:
def compiled_leak_result():
    
    max_nlags = len(cols) - 2
    extra_cols = paradox_cols + new_col_last_2 + new_col_last_3 + new_col_last_4 + new_col_last_5 \
                              + new_col_last_6
       
    extra_list = [paradox_cols, new_col_last_2, new_col_last_3, new_col_last_4, new_col_last_5,
                                new_col_last_6]
    
    train_leak = train[["ID", "target"] + cols + extra_cols]
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        train_leak[c] = get_log_pred(train_leak, cols, extra_list, i)
        leaky_cols.append(c)
        train_leak = train.join(
            train_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + extra_cols + leaky_cols+["compiled_leak", "nonzero_mean"]] ##changed 
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

In [129]:
train_leak, result = compiled_leak_result()

Processing lag 0
Leak values found in train 1376
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.5138333391635188
Processing lag 1
Leak values found in train 1986
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.2837324967824837
Processing lag 2
Leak values found in train 2376
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.158344347715661
Processing lag 3
Leak values found in train 2629
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.0599441159266052
Processing lag 4
Leak values found in train 2799
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.003768428505062
Processing lag 5
Leak values found in train 2948
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 0.9647709973956512
Processing lag 6
Leak values found in train 3070
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 0.9173110447464145
Processing lag 

In [130]:
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

best_score 0.6442714994927643 
best_lag 36


In [131]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [132]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]

In [133]:
train_leak = rewrite_compiled_leak(train_leak, best_lag+1)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000d6aaf2,38000000.0,38000000.0,38000000.0,0.0,38000000.0,0.0,38000000.0,0.0,0.0,...,38000000.0,0.0,38000000.0,0.0,38000000.0,38000000.0,0.0,0.0,0.0,38000000.0
1,000fbd867,600000.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,600000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0
2,0027d6b71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0028cbf45,0.0,0.0,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0
4,002a68644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
train_res = train_leak[["ID"]+leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak_paradox_and_newlast_2_6.csv', index=False)

In [135]:
def compiled_leak_result_test(max_nlags):
    extra_cols = paradox_cols + new_col_last_2 + new_col_last_3 + new_col_last_4 + new_col_last_5 \
                              + new_col_last_6
       
    extra_list = [paradox_cols, new_col_last_2, new_col_last_3, new_col_last_4, new_col_last_5,
                                new_col_last_6]    
    
    test_leak = test[["ID", "target"] + cols + extra_cols]
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        #test_leak[c] = _get_leak(test_leak, cols, i)
        
        test_leak[c] = get_log_pred(test_leak, cols, extra_list, i)
        leaky_cols.append(c)
        test_leak = test.join(
            test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + extra_cols + leaky_cols+["compiled_leak", "nonzero_mean"]]  #changed
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [136]:
test_leak, test_result = compiled_leak_result_test(max_nlags=38)

Processing lag 0
Leak values found in test 2957
Processing lag 1
Leak values found in test 4216
Processing lag 2
Leak values found in test 4965
Processing lag 3
Leak values found in test 5526
Processing lag 4
Leak values found in test 5943
Processing lag 5
Leak values found in test 6233
Processing lag 6
Leak values found in test 6437
Processing lag 7
Leak values found in test 6586
Processing lag 8
Leak values found in test 6749
Processing lag 9
Leak values found in test 6882
Processing lag 10
Leak values found in test 6993
Processing lag 11
Leak values found in test 7088
Processing lag 12
Leak values found in test 7161
Processing lag 13
Leak values found in test 7239
Processing lag 14
Leak values found in test 7296
Processing lag 15
Leak values found in test 7354
Processing lag 16
Leak values found in test 7400
Processing lag 17
Leak values found in test 7441
Processing lag 18
Leak values found in test 7471
Processing lag 19
Leak values found in test 7511
Processing lag 20
Leak values 

In [137]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
leaky_count,2957,4216,4965,5526,5943,6233,6437,6586,6749,6882,...,7715,7737,7757,7788,7813,7843,7890,7947,8029,8118


In [138]:
test_leak = rewrite_compiled_leak(test_leak, best_lag+1)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
test_res = test_leak[["ID"]+leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak_paradox_and_newlast_2_6.csv', index=False)

In [ ]:
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]

In [ ]:
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

In [ ]:
sub.head()

In [ ]:
#Paradox's col
#ordered_cols = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11',
#                '73687e512','aa164b93b','342e7eb03','cd24eae8a','8f3740670',
#                '2b2a10857','a00adf70e','3a48a2cd2','a396ceeb9','9280f3d04',
#                'fec5eaf1a','5b943716b','22ed6dba3','5547d6e11','e222309b0',
#                '5d3b81ef8','1184df5c2','2288333b4','f39074b55','a8b721722',
#                '13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
#                'b85fa8b27','2155f5e16']
#
#ordered_ids = ['a70090dc7','5b74875d4','ca6c4ea0d','242407fc6','7ff66b22b',
#               'f43a48254','c9573f4ac','c14136677','08f544828','e77e32574',
#               '61636fa41','4c0316d32','414871b56','2c518cd87','9eb93baef',
#               'd63b2307d','db9ddeb12','7e45f5bd5','680ecfac8','814f58e7b',
#               'eb850ef06','6edaf114d']
#
#ordered_indexes = []
#for oi in ordered_ids:
#    ordered_indexes.append(train[train.ID == oi].index[0])
#
#tmp = train.loc[ordered_indexes, ["ID","target"] + ordered_cols]
#print('shape', tmp.shape)
#tmp